In [68]:
import mysql.connector
from tabulate import tabulate


def mysql_request(request_table_name, params = None):
    connection = None
    try:
        connection = mysql.connector.connect(host='localhost',
                                     port=3306,
                                     user='root',
                                     password='123',
                                     db='sakila',)

        with connection.cursor(dictionary=True) as cursor:
            cursor.execute(request_table_name, params)
            result = cursor.fetchall()
            return result

    except mysql.connector.Error as ex:
        print(f"Database error [{ex.errno}]: {ex.msg}")
        return None

    finally:
        if connection and connection.is_connected():
            connection.close()
      

def get_categories():
    try:
        request_table_name = f"select category_id, name from category;"
        result = mysql_request(request_table_name)
        return result
    except mysql.connector.Error as er:
        print(f'Database error: {er.errno} : {er.msg}')

def print_categories():
    res = get_categories()
    if res:
        print("\n List of category:")
        print(tabulate(res, headers="keys", tablefmt="fancy_grid"))
    else:
        print("Error at getting categories.")


def get_films_of_category(category_id, operator=None, value=None):
    filter_clause = ""
    params = [category_id]
    # category_id = int(input('Input category ID: '))
    if operator and value:
        filter_clause = f"AND f.release_year {operator} %s"
        params.append(value)
        
    request_film = f'''select f.title, f.release_year, f.description,  GROUP_CONCAT(CONCAT(a.first_name, ' ', a.last_name) SEPARATOR ', ') AS actors
        from category c 
        join film_category fc 
        on c.category_id = fc.category_id 
        join film f 
        on fc.film_id = f.film_id
        join film_actor fa 
        on f.film_id = fa.film_id 
        join actor a 
        on fa.actor_id  = a.actor_id 
        where c.category_id = %s {filter_clause}
        GROUP BY f.film_id
        limit 10;
        '''
    return mysql_request(request_film, params)


def format_actors_for_new_lines(actors):
    if actors:
        actor_list = actors.split(', ')  
        formatted_actors = "\n".join(actor_list)  
        return formatted_actors
    return actors


def print_films(category_id, operator=None, value=None):
    res_film = get_films_of_category(category_id)
    if res_film:
        print("\n Films in your category:")
        for film in res_film:
            film['actors'] = format_actors_for_new_lines(film['actors'])
        print(tabulate(res_film, headers="keys", tablefmt="fancy_grid", 
                       maxcolwidths=[None, None, 40, None, 40]))  
    else:
        print("No data or error in your request")


def print_filter(category_id):
    valid_operators = ['=', '>', '<', '>=', '<=', '!=']
    use_filter = input("Would you add filter by release year? (y/n): ").strip().lower() == 'y'
    operator, value = None, None
    if use_filter:
        while True:
            operator = input(f"Choose operator ({', '.join(valid_operators)}): ").strip()
            if operator in valid_operators:
                break
            print("Error: Input correct operator!")

        while True:
            value = input("Input release year: ").strip()
            if value.isdigit():
                value = int(value)
                break
            print("Error: Input integer!")

    print_films(category_id, operator, value)



def main():
    print("Welcome to program for working with DB!")
    print_categories()

    while True:
        category_id = input("\nInput category ID (or 'exit' for finish work): ").strip()
        if category_id.lower() == 'exit':
            break
        if category_id.isdigit():
            print_filter(int(category_id))
        else:
            print("Error: Input correct category ID!")


if __name__ == "__main__":
    main()


Welcome to program for working with DB!

 List of category:
╒═══════════════╤═════════════╕
│   category_id │ name        │
╞═══════════════╪═════════════╡
│             1 │ Action      │
├───────────────┼─────────────┤
│             2 │ Animation   │
├───────────────┼─────────────┤
│             3 │ Children    │
├───────────────┼─────────────┤
│             4 │ Classics    │
├───────────────┼─────────────┤
│             5 │ Comedy      │
├───────────────┼─────────────┤
│             6 │ Documentary │
├───────────────┼─────────────┤
│             7 │ Drama       │
├───────────────┼─────────────┤
│             8 │ Family      │
├───────────────┼─────────────┤
│             9 │ Foreign     │
├───────────────┼─────────────┤
│            10 │ Games       │
├───────────────┼─────────────┤
│            11 │ Horror      │
├───────────────┼─────────────┤
│            12 │ Music       │
├───────────────┼─────────────┤
│            13 │ New         │
├───────────────┼─────────────┤
│           


Input category ID (or 'exit' for finish work):  3
Would you add filter by release year? (y/n):  y
Choose operator (=, >, <, >=, <=, !=):  =
Input release year:  2006



 Films in your category:
╒═════════════════════╤════════════════╤══════════════════════════════════════════╤═══════════════════╕
│ title               │   release_year │ description                              │ actors            │
╞═════════════════════╪════════════════╪══════════════════════════════════════════╪═══════════════════╡
│ BACKLASH UNDEFEATED │           2006 │ A Stunning Character Study of a Mad      │ CHRISTIAN AKROYD  │
│                     │                │ Scientist And a Mad Cow who must Kill a  │ SPENCER PECK      │
│                     │                │ Car in A Monastery                       │ CHRISTOPHER BERRY │
│                     │                │                                          │ SYLVESTER DERN    │
│                     │                │                                          │ DAN STREEP        │
│                     │                │                                          │ KEVIN GARLAND     │
│                     │               


Input category ID (or 'exit' for finish work):  exit
